# Data Cleaning & MySQL Integration for Power BI
## 📌 Overview
This notebook focuses on:

✅ Loading, cleaning, and structuring **540,000+ records**  
✅ Automating data import into MySQL instead of manual entry  
✅ Ensuring data integrity before visualizing in **Power BI**  

#  Install & Import Required Libraries

In [2]:
!pip install pandas mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 1.4 MB/s eta 0:00:0000:0100:01


In [11]:
!pip install pandas openpyxl mysql-connector-python

In [186]:
import mysql.connector
import pandas as pd

# Connect to MySQL

In [188]:
# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",  # Since your MySQL is running on your Mac
    user="root",  # Replace with your MySQL username
    password="password",  # Replace with your MySQL password
    database="RetailDB",  # Replace with your database name
    unix_socket="/tmp/mysql.sock",  # Use the socket file from the screenshot
    port=3306  # Default MySQL port
)

cursor = conn.cursor()
print("Successfully connected to MySQL!")

Successfully connected to MySQL!


In [3]:
cursor.execute("SHOW DATABASES;")
for db in cursor.fetchall():
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('RetailDB',)
('sys',)


In [5]:
cursor.execute("USE RetailDB;")
print("✅ Now using RetailDB!")

✅ Now using RetailDB!


# Load & Inspect Data

In [13]:
# Load the Excel file
file_path = "/Users/barbarawerobaobayi/Documents/Datasets/Online Retail.xlsx"

# Read the first sheet of the Excel file
df = pd.read_excel(file_path, engine="openpyxl")

# Display first 5 rows
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


#  Database Design & Table Creation

### **🛠 Database Schema**
We will create the following tables:
- **Customers** (CustomerID, Country)
- **Orders** (InvoiceNo, InvoiceDate, CustomerID)
- **Products** (StockCode, Description, UnitPrice)
- **OrderDetails** (InvoiceNo, StockCode, Quantity, UnitPrice)

This ensures **data integrity and efficient querying**.
You should have already done this in your MySQL.

In [7]:
cursor.execute("SHOW TABLES;")
for table in cursor.fetchall():
    print(table)

('Customers',)
('OrderDetails',)
('Orders',)
('Products',)


# Automated Data Loading with Python

### Data Cleaning & Transformation
- **✔ Fix Missing & Invalid Descriptions**
- **✔ Remove Zero-Priced Products**
- **✔ Fix Duplicate Orders**
- **✔ Handle Missing Foreign Keys**

In [29]:
customers = df[['CustomerID', 'Country']].drop_duplicates()
print(f" Extracted {len(customers)} unique customers!")

for _, row in customers.iterrows():
    cursor.execute("INSERT IGNORE INTO Customers (CustomerID, Country) VALUES (%s, %s)", 
                   (row["CustomerID"], row["Country"]))

conn.commit()
print("Inserted unique customers into MySQL!")

 Extracted 4389 unique customers!
Inserted unique customers into MySQL!


In [35]:
print(products.isna().sum())


StockCode        0
Description    960
UnitPrice        0
dtype: int64


In [81]:
# Define a list of invalid placeholders
invalid_descriptions = ["?", "??", "???", "missing", "?? missing", "????damages????", 
                        "???lost", "?display?", "?sold as sets?", "????missing", "???missing"]

# Replace NaN values with "No Description"
products['Description'] = products['Description'].fillna("No Description")

# Replace invalid placeholders with "Invalid Description"
products['Description'] = products['Description'].replace(invalid_descriptions, "Invalid Description")

# Ensure all descriptions are properly formatted as strings
products['Description'] = products['Description'].astype(str)

print(f"Extracted {len(products)} unique products after categorizing invalid descriptions!")


Extracted 16282 unique products after categorizing invalid descriptions!


In [95]:
products = products.dropna(subset=["Description"])
print(f"✅ Removed products with missing descriptions. Remaining products: {len(products)}")


✅ Removed products with missing descriptions. Remaining products: 17093


In [97]:
products = products[products["UnitPrice"] > 0]
print(f"✅ Removed products with zero price. Remaining products: {len(products)}")


✅ Removed products with zero price. Remaining products: 16282


In [99]:
print(products.isna().sum())  # Should show 0 NaNs
print(products[products["UnitPrice"] == 0])  # Should return an empty DataFrame
print(products.dtypes)  # Ensure correct data types


StockCode      0
Description    0
UnitPrice      0
dtype: int64
Empty DataFrame
Columns: [StockCode, Description, UnitPrice]
Index: []
StockCode       object
Description     object
UnitPrice      float64
dtype: object


In [101]:
for _, row in products.iterrows():
    cursor.execute("INSERT IGNORE INTO Products (StockCode, Description, UnitPrice) VALUES (%s, %s, %s)", 
                   (row["StockCode"], row["Description"], row["UnitPrice"]))

conn.commit()
print("✅ Successfully inserted cleaned product data into MySQL!")


✅ Successfully inserted cleaned product data into MySQL!


In [105]:
print(orders["InvoiceNo"].duplicated().sum())  # Count duplicates
print(orders[orders.duplicated(subset="InvoiceNo", keep=False)].head(10))  # Show sample duplicate invoices


516009
  InvoiceNo         InvoiceDate CustomerID
0    536365 2010-12-01 08:26:00    17850.0
1    536365 2010-12-01 08:26:00    17850.0
2    536365 2010-12-01 08:26:00    17850.0
3    536365 2010-12-01 08:26:00    17850.0
4    536365 2010-12-01 08:26:00    17850.0
5    536365 2010-12-01 08:26:00    17850.0
6    536365 2010-12-01 08:26:00    17850.0
7    536366 2010-12-01 08:28:00    17850.0
8    536366 2010-12-01 08:28:00    17850.0
9    536367 2010-12-01 08:34:00    13047.0


In [107]:
orders = orders.drop_duplicates(subset="InvoiceNo")
print(f"Extracted {len(orders)} unique orders after removing duplicates!")

Extracted 25900 unique orders after removing duplicates!


In [109]:
for _, row in orders.iterrows():
    cursor.execute("INSERT IGNORE INTO Orders (InvoiceNo, InvoiceDate, CustomerID) VALUES (%s, %s, %s)", 
                   (row["InvoiceNo"], row["InvoiceDate"], row["CustomerID"]))

conn.commit()
print("Inserted unique order records into MySQL!")


Inserted unique order records into MySQL!


In [113]:
# Find missing StockCodes
missing_stockcodes = order_details[~order_details["StockCode"].isin(products["StockCode"])]

print(f"❌ Missing StockCodes: {len(missing_stockcodes)}")
print(missing_stockcodes.head(10))  # Display first few missing StockCodes


❌ Missing StockCodes: 134
      InvoiceNo StockCode  Quantity  UnitPrice
1970     536545     21134         1        0.0
1987     536549    85226A         1        0.0
1988     536550     85044         1        0.0
2024     536552     20950         1        0.0
2026     536554     84670        23        0.0
7187     536995     35951        57        0.0
7193     537001     21653        -6        0.0
19628    537875     20849         1        0.0
19631    537878    72803B         1        0.0
21782    538133    85018C         3        0.0


In [117]:
order_details = order_details[order_details["StockCode"].isin(products["StockCode"])]
print(f" Removed {len(missing_stockcodes)} invalid OrderDetails rows. Remaining: {len(order_details)}")

 Removed 134 invalid OrderDetails rows. Remaining: 541775


In [119]:
# Create placeholder products for missing StockCodes
missing_products = missing_stockcodes[["StockCode"]].drop_duplicates()
missing_products["Description"] = "Unknown Product"
missing_products["UnitPrice"] = 0.0  # Default price

# Insert missing products into MySQL
for _, row in missing_products.iterrows():
    cursor.execute("INSERT IGNORE INTO Products (StockCode, Description, UnitPrice) VALUES (%s, %s, %s)", 
                   (row["StockCode"], row["Description"], row["UnitPrice"]))

conn.commit()
print(f"Inserted {len(missing_products)} missing products into MySQL!")


Inserted 132 missing products into MySQL!


In [121]:
# Check if any missing StockCodes remain
missing_stockcodes = order_details[~order_details["StockCode"].isin(products["StockCode"])]
print(f"❌ Missing StockCodes after fix: {len(missing_stockcodes)}")


❌ Missing StockCodes after fix: 0


In [125]:
for _, row in order_details.iterrows():
    cursor.execute("INSERT INTO OrderDetails (InvoiceNo, StockCode, Quantity, UnitPrice) VALUES (%s, %s, %s, %s)", 
                   (row["InvoiceNo"], row["StockCode"], row["Quantity"], row["UnitPrice"]))

conn.commit()
print("Successfully inserted order details into MySQL!")

Successfully inserted order details into MySQL!


In [127]:
# Check row counts for all tables
tables = ["Customers", "Products", "Orders", "OrderDetails"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table};")
    row_count = cursor.fetchone()[0]
    print(f"✅ {table}: {row_count} rows")


✅ Customers: 4373 rows
✅ Products: 3958 rows
✅ Orders: 25900 rows
✅ OrderDetails: 1085520 rows


In [161]:
cursor.execute("""
    SELECT InvoiceNo, StockCode, COUNT(*)
    FROM OrderDetails
    GROUP BY InvoiceNo, StockCode
    HAVING COUNT(*) > 1;
""")
duplicates = cursor.fetchall()

print(f"❌ Duplicate rows in OrderDetails: {len(duplicates)}")

❌ Duplicate rows in OrderDetails: 531089


In [163]:
cursor.execute("DELETE FROM OrderDetails;")
conn.commit()
print("✅ Cleared OrderDetails table!")

✅ Cleared OrderDetails table!


In [171]:
order_details = order_details.drop_duplicates(subset=["InvoiceNo", "StockCode"])
print(f" Reloading {len(order_details)} unique OrderDetails rows!")

for _, row in order_details.iterrows():
    cursor.execute("INSERT INTO OrderDetails (InvoiceNo, StockCode, Quantity, UnitPrice) VALUES (%s, %s, %s, %s)", 
                   (row["InvoiceNo"], row["StockCode"], row["Quantity"], row["UnitPrice"]))

conn.commit()
print(" Successfully reloaded clean OrderDetails into MySQL!")

 Reloading 531091 unique OrderDetails rows!
 Successfully reloaded clean OrderDetails into MySQL!


# Final Checks & Validation

In [172]:
# Check row counts for all tables
tables = ["Customers", "Products", "Orders", "OrderDetails"]

for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table};")
    row_count = cursor.fetchone()[0]
    print(f"{table}: {row_count} rows")

Customers: 4373 rows
Products: 3958 rows
Orders: 25900 rows
OrderDetails: 531091 rows
